# Imports

In [41]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

import tensorflow as tf
from keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D, ReLU, Flatten, BatchNormalization

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, accuracy_score

In [62]:
model_directory = "/content/gdrive/MyDrive/GW_SOP/Code/debugging/1D-CNN Multi-level/Signal_HIDDEN_in_strain_data/ALIGO_noise/Test 5/data/"
data_directory = "/content/gdrive/MyDrive/GW_SOP/Data/Marginal events/"
# results_directory = "/content/gdrive/MyDrive/GW_SOP/Code/debugging/1D-CNN Multi-level/Signal_HIDDEN_in_strain_data/ALIGO_noise/Test 5/data/Marginal Event Results/bar plots/1e19/"
results_directory = "/content/gdrive/MyDrive/GW_SOP/Code/debugging/1D-CNN Multi-level/Signal_HIDDEN_in_strain_data/ALIGO_noise/Test 5/data/Marginal Event Results/bar plots/1e18/"

In [63]:
marginal_events =  ["GW190531_023648  GWTC-2.1-marginal",
                    "170405  GWTC-1-marginal",
                    "170208  GWTC-1-marginal",
                    "200114_020818  O3_IMBH_marginal",
                    "170705  GWTC-1-marginal",
                    "200219_201407  GWTC-3-marginal",
                    "191223_014159  O3_IMBH_marginal",
                    "170412  GWTC-1-marginal",
                    "200214_224526  GWTC-3-marginal",
                    "190924_232654  O3_IMBH_marginal",
                    "170616  GWTC-1-marginal",
                    "GW200105_162426  GWTC-3-marginal",
                    "GW191118_212859  GWTC-3-marginal",
                    "151008  GWTC-1-marginal",
                    "191225_215715  O3_IMBH_marginal",
                    "200121_031748  GWTC-3-marginal",
                    "170720  GWTC-1-marginal",
                    "170219  GWTC-1-marginal",
                    "GW200311_103121  GWTC-3-marginal",
                    "151012.2  GWTC-1-marginal",
                    "161202  GWTC-1-marginal",
                    "151116  GWTC-1-marginal",
                    "GW190426_152155  GWTC-2.1-marginal",
                    "170630  GWTC-1-marginal",
                    "161217  GWTC-1-marginal",
                    "170423  GWTC-1-marginal",
                    "GW200201_203549  GWTC-3-marginal"]

# Testing on Real data

## Level 1

In [64]:
marginal_df = pd.read_csv(data_directory + "Original_signals.csv", header=None)
marginal = marginal_df.values.astype(float)

In [65]:
X = marginal
print(X.shape)

(27, 16384)


In [66]:
X *= 1e18
print(X)

# X *= 1e19
# print(X)

[[-0.18716659 -0.13814955 -0.10986291 ...  0.04578542  0.09510752
   0.11681538]
 [ 0.89122314  0.89725775  0.88749534 ... -0.47559963 -0.47960982
  -0.47806027]
 [-1.22831225 -1.22288272 -1.25143445 ... -2.55568957 -2.5862049
  -2.58620579]
 ...
 [-1.95298871 -1.9697331  -1.94882194 ... -0.60242158 -0.62078926
  -0.62580659]
 [-0.70815089 -0.71240508 -0.70967189 ... -0.26686678 -0.26254276
  -0.27034402]
 [ 0.95262939  0.97120658  0.98883936 ... -0.14334126 -0.14709983
  -0.153142  ]]


In [67]:
X_test_l1 = np.expand_dims(X, axis=-1)
print(X_test_l1.shape)

(27, 16384, 1)


In [68]:
model_l1 = load_model(model_directory + "model_trial_1_level_1.h5")

In [69]:
pred_l1 = model_l1.predict(X_test_l1)

In [70]:
y_pred_l1 = np.argmax(pred_l1, axis = 1)
X_test_l2 = []
name = []

for i in range(len(X_test_l1)):

  if y_pred_l1[i] == 1:
    X_test_l2.append(X_test_l1[i])

    name.append(marginal_events[i])

print(f"Total no. of data samples in level 1: {len(X)}")
print(f"No. of data samples predicted as signal: {len(X_test_l2)}")
print(f"No. of data samples predicted as noise: {len(X) - len(X_test_l2)}")
print(f"No. of data samples sent to level 2 model: {len(X_test_l2)}")

Total no. of data samples in level 1: 27
No. of data samples predicted as signal: 25
No. of data samples predicted as noise: 2
No. of data samples sent to level 2 model: 25


In [71]:
pos_probs = pred_l1[:,1]

In [72]:
classes = ["Noise", "Signal"]

for i in range(len(X_test_l1)):
  prob = list(pred_l1[i])
  fig = plt.figure(figsize =(4.5,4.5))

  plt.bar(classes, prob, color ='maroon',
          width = 0.4)
  
  plt.ylabel("Probability")

  plt.title(marginal_events[i])

  plt.savefig(results_directory + "Level 1/" + str(i) + ".png")
  plt.close()
  # plt.show()

np.savetxt(results_directory + "Level 1/probabilities.csv", pred_l1, delimiter=",")

## Level 2

In [73]:
X_test_l2 = np.array(X_test_l2)
print(X_test_l2.shape)

(25, 16384, 1)


In [74]:
model2 = load_model(model_directory + "model_trial_1_level_2.h5")

In [75]:
pred_l2 = model2.predict(X_test_l2)

In [76]:
y_pred_l2 = np.argmax(pred_l2, axis = 1)
bbh = 0
bns = 0

for i in range(len(y_pred_l2)):
    if y_pred_l2[i] == 0:
        bbh += 1

    else:
        bns += 1

print(f"Total no. of data samples in level 2: {len(X_test_l2)}")
print(f"No. of data samples predicted as BBH signals: {bbh}")
print(f"No. of data samples predicted as BNS signals: {bns}")

Total no. of data samples in level 2: 25
No. of data samples predicted as BBH signals: 25
No. of data samples predicted as BNS signals: 0


In [77]:
pos_probs = pred_l2[:,1]

In [78]:
classes = ["BBH", "BNS"]

for i in range(len(X_test_l2)):
  prob = list(pred_l2[i])
  fig = plt.figure(figsize =(4.5,4.5))

  plt.bar(classes, prob, color ='maroon',
          width = 0.4)
  
  plt.ylabel("Probability")

  plt.title(name[i])

  plt.savefig(results_directory + "Level 2/" + str(i) + ".png")
  plt.close()
  # plt.show()

np.savetxt(results_directory + "Level 2/probabilities.csv", pred_l2, delimiter=",")

## Overall

In [79]:
y_true = []
probs = []
tmp = []
j = 0

for i in range(len(X)):

  if y_pred_l1[i] == 0:
    tmp.append(pred_l1[i, 0])
    tmp.append(pred_l1[i, 1])
    tmp.append(pred_l1[i, 1])
    probs.append(tmp)
    tmp = []
  
  else:
    tmp.append(pred_l1[i, 0])
    tmp.append(pred_l1[i, 1]*pred_l2[j, 0])
    tmp.append(pred_l1[i, 1]*pred_l2[j, 1])

    j += 1
    probs.append(tmp)
    tmp = []

probs = np.array(probs)
y_pred = np.argmax(probs, axis = 1)

In [80]:
classes = ["Noise", "BBH", "BNS"]

for i in range(len(X)):
  prob = list(probs[i])
  fig = plt.figure(figsize =(4.5,4.5))

  plt.bar(classes, prob, color ='maroon',
          width = 0.4)
  
  plt.ylabel("Probability")
  
  plt.title(marginal_events[i])

  plt.savefig(results_directory + "Overall/" + str(i) + ".png")
  plt.close()
  # plt.show()

np.savetxt(results_directory + "Overall/probabilities.csv", probs, delimiter=",")